# 1. Run pro con service on arguments collected from the crowd 

We will start with reading the arguments from a csv file, define the topic, and create a debater_api object

In [ ]:
import os
from debater_python_api.api.debater_api import DebaterApi
from debater_python_api.api.clients.narrative_generation_client import Polarity
import csv


api_key = os.environ['API_KEY']
debater_api = DebaterApi(api_key)
arguments_file = 'wealth_test_set.csv'
topic = 'It is time to redistribute the wealth'

with open(arguments_file, encoding='utf8') as csv_file:
    reader = csv.DictReader(csv_file)
    examples = list(reader)
    arguments = [example['sentence'] for example in examples]
    labels = [example['label'] for example in examples]

new we are ready to run the pro con service. The pro con service gets a list of dictionaries where each dictionary contains two keys: an argument and a topic. (e.g. arg_topic_dicts = [{'sentence': 'first sentence', 'topic': 'first topic'}, {'sentence': 'second sentence', 'topic': 'second topic'}] 

The service returns a list of floats in the range [-1, 1], where each float is the prediction (score) for the corresponding sentence and topic. The sign of the score indicated if the service predicts that the argument supports the motion (positive score), or contests the topic (negative score). The absolute value of the score indicated the confidence of the service. A low value indicates low confidence, while a high value indicates strong confidenec.

Please write a code that create a pro_con_client by calling the method get_pro_con_client() of the debater_api object.
Create a list with the arguments we got from the crowd while using the topic we defined, and run the pro con client on the list, while storing the results at a list with the name pro_con_scores.


In [ ]:
pro_con_client = debater_api.get_pro_con_client()
sentence_topic_dicts = [{'sentence': argument, 'topic': topic} for argument in arguments]
pro_con_scores = pro_con_client.run(sentence_topic_dicts)

# 2. Use pro con results
Now, we can use the service results and measure it accuracy. We will compare the service prediction to the labels created by human annotators. The label field contains tree values: '1' for supporting arguments, '-1' for contesting arguments, and '0' for nuetral or mixed arguments.

In [ ]:
argument_score_label_list = list(zip(arguments, pro_con_scores, labels))

predicted_pro = [triple for triple in argument_score_label_list if triple[1] > 0]
predicted_pro_labeled_pro = [triple for triple in predicted_pro if triple[2] == '1']
print('predicted pro accuracy {}'.format(len(predicted_pro_labeled_pro) / len(predicted_pro)))

predicted_con = [triple for triple in argument_score_label_list if triple[1] < 0]
predicted_con_labeled_con = [triple for triple in predicted_con if triple[2] == '-1']
print('predicted con accuracy {}'.format(len(predicted_con_labeled_con) / len(predicted_con)))


# 3. Generate coverage - precision curve

as we saw in the previous run, although most of the time the prediction is right, still we have about 30% errors. 

In [ ]:
from wealth_util import calculate_statistics, plot_graph

pro_con_statistics = calculate_statistics('./wealth_test_set.csv', pro_con_scores)
plot_graph(examples_list=[pro_con_statistics], labels=['prod'])

# 4. Train pro - con service

# 5. Generate narrative using the fine tuned model 